In [10]:
import pandas as pd
import numpy as np
import os
import glob

from datetime import date
from datetime import timedelta
import os

tdy = str(date.today().isoformat()).replace('-','')

In [11]:
aptaprice = pd.read_csv("D:\\시세모니터링\\융합분석\\과거 시가총액 조회\\dat_all_20201113.csv", index_col=0)
# aptaprice = pd.read_csv("D:\\시세모니터링\\융합분석\\과거 시가총액 조회\\dat_all_2020921.csv", index_col=0)
aptaprice = aptaprice.rename(columns={"조사일" : 'date'})
# aptaprice.index = aptaprice["date"]
# aptaprice = aptaprice.drop(columns=["date"])
aptaprice["date"] = aptaprice["date"].astype(str).str[:4] + "-" + aptaprice["date"].astype(str).str[4:6] + "-" + aptaprice["date"].astype(str).str[-2:]
sgg_lst = pd.unique(aptaprice.지역명)

# aptaprice["date_time"] = pd.to_datetime(aptaprice["date"], format='%Y-%m-%d')

# 기간 설정
# aptaprice = aptaprice[aptaprice["date_time"] > '2006-12-31']
aptaprice

,date,지역명,aptprice_all_sum
0,1997-01-15,전국,1.279720e+10
1,1997-02-15,전국,1.295263e+10
2,1997-03-15,전국,1.307766e+10
3,1997-04-15,전국,1.316765e+10
4,1997-05-15,전국,1.332292e+10
...,...,...,...
33960,2020-11-13,파주시,2.310988e+09
33961,2020-11-13,평택시,2.341185e+09
33962,2020-11-13,포천시,1.843444e+08
33963,2020-11-13,하남시,3.526432e+09


In [17]:
from tqdm import tqdm_notebook
import datetime
path = "D:/시세모니터링/융합분석/DATA/20201113/"
# path = "D:/시세모니터링/융합분석/DATA/" + datetime.datetime.now().isoformat()[:10].replace("-","")
# path = "D:/시세모니터링/융합분석/DATA/" + (datetime.datetime.now() - datetime.timedelta(days=1)).isoformat()[:10].replace("-","")

file = glob.glob(path + "\\*.csv")

## 결과저장용 dataframe 생성 (index = 시군구, 컬럼수 : 크롤링 데이터 수 대로 0 을 채워넣음)
# empt_lst_idx = pd.DataFrame(np.zeros(74 * len(file)).reshape(74,-1), columns = [i.split('.csv')[0].split('\\')[1] for i in file], index = sgg_lst)
empt_lst_idx = pd.DataFrame(np.zeros(len(pd.unique(aptaprice.지역명)) * len(file)).reshape(len(pd.unique(aptaprice.지역명)),-1), columns = [i.split('.csv')[0].split('\\')[1] for i in file], index = sgg_lst)


for i in tqdm_notebook(file):
    list_temp = pd.read_csv(i)
    list_temp = list_temp.iloc[:,:2]
    list_temp = list_temp.rename(columns={"Date" : 'date'})
    
    try :
        list_temp.iloc[:,1] = list_temp.iloc[:,1].str.replace(",","").astype(float)
    except :
        pass
    
    apt_merge = pd.merge(aptaprice, list_temp, how = 'inner')
#     apt_merge.to_csv(path+"apt_gu_" + i.split("\\")[1].split(".csv")[0]+ '.csv')
    
    for num,sgg_nm in enumerate(sgg_lst):
        corr_gu = apt_merge[apt_merge["지역명"] == sgg_nm].sort_values("date")
        res = corr_gu.iloc[:,2:].corr()["aptprice_all_sum"].sort_values(ascending=True)[0]
        
        #결과저장용 df에 insert수행
        empt_lst_idx.loc[empt_lst_idx.index == sgg_nm, corr_gu.columns[3]] = res
        
#         if ((res > 0.7) | (res < -0.7)) & (len(corr_gu) > 1000):
#             # 상관관계 0.5 이상, 건수 500건 이상
# #             apt_merge.to_csv(path+"apt_gu_" + i.split("\\")[1].split(".csv")[0]+ '.csv')
        print(str(num) + "/ 25")
        print("# " + sgg_nm + " 시가총액과 " + i.split("\\")[1].split(".csv")[0] + " 의 상관관계")
        print("# 대상 Row수 : " + str(len(corr_gu)) + "건")
        print("# 관계 : " + str(res))
        print("\n")
                        
#         elif ((res < 0.5) | (res > -0.5)) :
#             print("# " + sgg_nm + " 시가총액과 " + i.split("\\")[1].split(".csv")[0])
#             print("# 상관관계 낮음 : " + str(res))
#             print("# 제외 대상 Row수 : " + str(len(corr_gu)) + "건")
#             print("\n")
            
#         elif len(corr_gu) < 500:
#             print("# " + sgg_nm + " 시가총액과 " + i.split("\\")[1].split(".csv")[0])
#             print("# 제외 대상 Row수 : " + str(len(corr_gu)) + "건")
#             print("\n")

# 결과저장용 df CSV 저장
empt_lst_idx.T.to_csv("sgg_gu_cor_res_"+ tdy + ".csv", encoding="CP949")

c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


0/ 25
# 전국 시가총액과 aud-chf 의 상관관계
# 대상 Row수 : 1123건
# 관계 : -0.6886839670922048


1/ 25
# 강원도 시가총액과 aud-chf 의 상관관계
# 대상 Row수 : 1123건
# 관계 : -0.7031313978539092


2/ 25
# 경기도 시가총액과 aud-chf 의 상관관계
# 대상 Row수 : 1123건
# 관계 : -0.6396719141109203


3/ 25
# 경상남도 시가총액과 aud-chf 의 상관관계
# 대상 Row수 : 1123건
# 관계 : -0.7097200230458207


4/ 25
# 경상북도 시가총액과 aud-chf 의 상관관계
# 대상 Row수 : 1115건
# 관계 : -0.7291399970919955


5/ 25
# 광주광역시 시가총액과 aud-chf 의 상관관계
# 대상 Row수 : 1123건
# 관계 : -0.7502989914026461


6/ 25
# 대구광역시 시가총액과 aud-chf 의 상관관계
# 대상 Row수 : 1123건
# 관계 : -0.7781587778030438


7/ 25
# 대전광역시 시가총액과 aud-chf 의 상관관계
# 대상 Row수 : 1123건
# 관계 : -0.6885821286418948


8/ 25
# 부산광역시 시가총액과 aud-chf 의 상관관계
# 대상 Row수 : 1123건
# 관계 : -0.7453351019302611


9/ 25
# 서울특별시 시가총액과 aud-chf 의 상관관계
# 대상 Row수 : 1123건
# 관계 : -0.6615568418048503


10/ 25
# 인천광역시 시가총액과 aud-chf 의 상관관계
# 대상 Row수 : 1123건
# 관계 : -0.6804868670883202


11/ 25
# 전라북도 시가총액과 aud-chf 의 상관관계
# 대상 Row수 : 1123건
# 관계 : -0.646887864569426


12/ 25
# 충청남도 시가총액과 aud-c

In [18]:
empt_lst_idx.head(2)

,aud-chf,CD(91일),CMA(수시형),cny-usd,copper,CP(91일),crude-oil,eur-usd,gold,hang-sen-40,...,장내파생상품 _거래 예수금,"콜금리(1일, 전체거래)","콜금리(1일, 중개회사거래)",통안증권(1년),통안증권(2년),통안증권(91일),투자자예탁금_(장내파생상품 거래예수금제외),회사채(3년)(BBB-),"회사채(3년,AA-)","회사채(AA-, 민평)수익률"
전국,-0.688684,-0.842526,-0.895833,0.385343,0.665333,-0.810894,0.388283,0.280209,0.880483,0.854220,...,0.878456,-0.840422,-0.836040,-0.856275,-0.860581,-0.761938,0.859312,-0.430520,-0.817180,-0.758154
강원도,-0.703131,-0.859089,-0.897490,0.554359,0.646172,-0.836979,0.386746,0.227853,0.889642,0.853977,...,0.846784,-0.862655,-0.860015,-0.871858,-0.877796,-0.836260,0.828462,-0.421302,-0.841625,-0.809796


In [5]:
empt_lst_idx.T.to_csv("sgg_gu_cor_res_"+ tdy + ".csv", encoding="CP949")
res_corr_all = pd.read_csv("sgg_gu_cor_res_"+ tdy + ".csv", encoding="CP949", index_col=0)
res_corr_all.head()

,전국,강원도,경기도,경상남도,경상북도,광주광역시,대구광역시,대전광역시,부산광역시,서울특별시,...,동두천시,안성시,오산시,포천시,가평군,양주시,화성시,여주시,양평군,연천군
aud-chf,-0.688621,-0.703106,-0.639573,-0.710030,-0.729028,-0.750299,-0.778149,-0.688531,-0.745323,-0.661469,...,-0.532539,-0.609337,-0.611227,-0.525419,-0.599124,-0.553226,-0.695406,-0.575463,-0.522450,-0.528265
CD(91일),-0.842554,-0.859078,-0.825615,-0.874327,-0.882491,-0.868922,-0.866534,-0.863604,-0.866866,-0.793295,...,-0.812270,-0.858425,-0.849415,-0.808027,-0.827069,-0.811810,-0.829253,-0.828232,-0.806477,-0.741953
CMA(수시형),-0.895881,-0.897483,-0.891934,-0.910238,-0.917333,-0.896519,-0.894033,-0.912418,-0.896742,-0.853015,...,-0.845543,-0.899132,-0.896625,-0.871907,-0.856171,-0.868417,-0.863175,-0.878360,-0.859296,-0.794316
cny-usd,0.385572,0.554410,0.389989,0.582891,0.536669,0.511563,0.401597,0.464231,0.459990,0.223811,...,0.815127,0.738524,0.699636,0.729857,0.767539,0.659245,0.365396,0.716702,0.801678,0.803569
copper,0.665453,0.646187,0.717376,0.637677,0.609265,0.582591,0.544426,0.656796,0.584172,0.650987,...,0.702322,0.676852,0.698947,0.726403,0.600861,0.690074,0.493235,0.580968,0.586245,0.552947


In [6]:
# cor_relation 기준선
corr_var = 0.75

res_index_nm1 = res_corr_all['전국'][abs(res_corr_all['전국']) > corr_var].reset_index()["index"]
res_index_nm2 = res_corr_all['서울특별시'][abs(res_corr_all['서울특별시']) > corr_var].reset_index()["index"]
res_index_nm3 = res_corr_all['경기도'][abs(res_corr_all['경기도']) > corr_var].reset_index()["index"]
res_index_nm4 = res_corr_all['강남구'][abs(res_corr_all['강남구']) > corr_var].reset_index()["index"]

# res_index_nm = pd.merge(res_index_nm1, res_index_nm2, how = 'outer')
# res_index_nm = pd.merge(res_index_nm, res_index_nm3, how = 'inner')
# len(res_index_nm["index"].values)
res_index_nm = pd.unique(pd.concat([res_index_nm1, res_index_nm2,res_index_nm3, res_index_nm4]))
res_index_nm

array(['CD(91일)', 'CMA(수시형)', 'CP(91일)', 'gold', 'hang-sen-40',
       'KORIBOR(12개월)', 'KORIBOR(3개월)', 'KORIBOR(6개월)', 'kospi',
       'KOSPI_시가총액', 'kr_bs_rate', 'MMF(7일)', 'nq-100', 'smallcap-2000',
       'u.s.-10-year-bond-yield', 'u.s.-30-year-bond-yield',
       'u.s.-5-year-bond-yield', 'us-30', 'us-spx-500', 'usd-cny',
       '국고채(10년)', '국고채(1년)', '국고채(20년)', '국고채(30년)', '국고채(3년)',
       '국고채(5년)', '국민주택채권1종(5년)', '대고객 환매 _조건부 채권(RP) 매도잔고', '산금채(1년)',
       '장내파생상품 _거래 예수금', '콜금리(1일, 전체거래)', '콜금리(1일, 중개회사거래)', '통안증권(1년)',
       '통안증권(2년)', '통안증권(91일)', '투자자예탁금_(장내파생상품 거래예수금제외)', '회사채(3년,AA-)',
       '회사채(AA-, 민평)수익률', 'KOSDAQ_거래대금'], dtype=object)

In [7]:
len(res_index_nm)

39

In [8]:
# res_list = [
#             "u.s.-30-year-bond-yield",
#             "hang-sen-40",
#             "smallcap-2000",
#             "gold",
#             "silver",
#             "nq-100",
#             'aud-chf'
#            ]

res_list = res_index_nm

In [9]:
reslt = aptaprice

for a in file:
    if (a.split("\\")[1].split(".csv")[0] in res_list) == True:
        temp_res = pd.read_csv(a)
        temp_res = temp_res.iloc[:,:2]
        temp_res = temp_res.rename(columns={"Date" : 'date'})
        
        reslt = pd.merge(reslt, temp_res, how = 'inner')
reslt.to_csv("merge_ext_FR2007to2020_" + tdy + ".csv", index=False)
reslt

,date,지역명,aptprice_all_sum,CD(91일),CMA(수시형),CP(91일),gold,hang-sen-40,KORIBOR(12개월),KORIBOR(3개월),...,산금채(1년),장내파생상품 _거래 예수금,"콜금리(1일, 전체거래)","콜금리(1일, 중개회사거래)",통안증권(1년),통안증권(2년),통안증권(91일),투자자예탁금_(장내파생상품 거래예수금제외),"회사채(3년,AA-)","회사채(AA-, 민평)수익률"
0,2012-12-07,전국,1.863189e+11,2.87,2.70,2.98,1704.0,22191.17,2.94,2.83,...,2.86,8192732.0,2.76,2.76,2.790,2.810,2.770,17090080.0,3.250,3.230
1,2012-12-07,강원도,2.054871e+09,2.87,2.70,2.98,1704.0,22191.17,2.94,2.83,...,2.86,8192732.0,2.76,2.76,2.790,2.810,2.770,17090080.0,3.250,3.230
2,2012-12-07,경기도,5.707744e+10,2.87,2.70,2.98,1704.0,22191.17,2.94,2.83,...,2.86,8192732.0,2.76,2.76,2.790,2.810,2.770,17090080.0,3.250,3.230
3,2012-12-07,경상남도,7.137563e+09,2.87,2.70,2.98,1704.0,22191.17,2.94,2.83,...,2.86,8192732.0,2.76,2.76,2.790,2.810,2.770,17090080.0,3.250,3.230
4,2012-12-07,경상북도,3.223587e+09,2.87,2.70,2.98,1704.0,22191.17,2.94,2.83,...,2.86,8192732.0,2.76,2.76,2.790,2.810,2.770,17090080.0,3.250,3.230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26931,2020-11-06,파주시,2.306063e+09,0.66,0.24,1.11,1951.7,25712.97,0.91,0.64,...,0.86,10892055.0,0.51,0.51,0.711,0.857,0.643,51899035.0,2.249,1.514
26932,2020-11-06,평택시,2.337911e+09,0.66,0.24,1.11,1951.7,25712.97,0.91,0.64,...,0.86,10892055.0,0.51,0.51,0.711,0.857,0.643,51899035.0,2.249,1.514
26933,2020-11-06,포천시,1.843444e+08,0.66,0.24,1.11,1951.7,25712.97,0.91,0.64,...,0.86,10892055.0,0.51,0.51,0.711,0.857,0.643,51899035.0,2.249,1.514
26934,2020-11-06,하남시,3.513690e+09,0.66,0.24,1.11,1951.7,25712.97,0.91,0.64,...,0.86,10892055.0,0.51,0.51,0.711,0.857,0.643,51899035.0,2.249,1.514
